In [1]:
import os
from openai import OpenAI

client = OpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
)



# 2.1 原则一：编写清晰、具体的指令
您应该通过提供尽可能清晰和具体的指令来表达您希望模型执行的操作。这将引导模型给出正确的输出，并降低您得到无关或不正确响应的可能性。清晰的指令不意味着必须简短，在许多情况下，更长的 Prompt 实际上更清晰，且提供了更多上下文，也就可能产生更详细更相关的输出。

## 2.1.1 使用分隔符清晰地表示输入的不同部分

分隔符可以是：```，""，<>，:，<tag> </tag>等。

您可以使用任何明显的标点符号将特定的文本部分与 Prompt 的其余部分分开。标记的形式不限，只需要让模型明确知道这是一个单独部分。使用分隔符可以有效避免提示词注入( Prompt injection )。提示词注入是指如果允许用户将某些输入添加到（开发者预定义的） Prompt 中，则所提供的指令可能会与开发者想要执行的操作相冲突，从而使 LLM 遵循用户输入的指令，而非执行开发者预期的操作。即，输入里面可能包含其他指令，会覆盖掉您的指令。对此，使用分隔符是一个不错的策略。

In [2]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message.content

In [3]:
# 中文版见下一个 cell
text = f"""
You should express what you want a model to do by 
providing instructions that are as clear and 
specific as you can possibly make them. 
This will guide the model towards the desired output, 
and reduce the chances of receiving irrelevant 
or incorrect responses. Don't confuse writing a 
clear prompt with writing a short prompt.  
In many cases, longer prompts provide more clarity 
and context for the model, which can lead to 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks 
into a single sentence.
```{text}```
"""

print(f'prompt:{prompt}')
response = get_completion(prompt)
print(response)

prompt:
Summarize the text delimited by triple backticks 
into a single sentence.
```
You should express what you want a model to do by 
providing instructions that are as clear and 
specific as you can possibly make them. 
This will guide the model towards the desired output, 
and reduce the chances of receiving irrelevant 
or incorrect responses. Don't confuse writing a 
clear prompt with writing a short prompt.  
In many cases, longer prompts provide more clarity 
and context for the model, which can lead to 
more detailed and relevant outputs.
```

Clear and specific instructions are essential for guiding a model towards the desired output and reducing the chances of irrelevant or incorrect responses, with longer prompts often providing more clarity and context for more detailed and relevant outputs.


In [4]:
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。
不要将写清晰的提示词与写简短的提示词混淆。
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""
# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""
# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

提供清晰、具体的指示可以引导模型朝向所需的输出，避免无关或不正确的响应。更长的提示词可以提供更多清晰度和上下文信息，导致更详细和相关的输出。


## 2.1.2 寻求结构化的输出

输出可以是 Json、HTML 等格式。

第二个策略是要求生成一个结构化的输出，这可以使模型的输出更容易被我们解析

In [6]:
prompt = f"""
Generate a list of three made-up book titles along  
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

[
    {
        "book_id": 1,
        "title": "The Midnight Garden",
        "author": "Elena Rivers",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "Echoes of the Past",
        "author": "Nathan Black",
        "genre": "Mystery"
    },
    {
        "book_id": 3,
        "title": "Whispers in the Wind",
        "author": "Samantha Reed",
        "genre": "Romance"
    }
]


In [7]:
prompt = f"""
请生成包括书名、作者和类别的三本虚构书籍清单，
并以 JSON 格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

[
    {
        "book_id": 1,
        "title": "The Night Circus",
        "author": "Erin Morgenstern",
        "genre": "Fantasy"
    },
    {
        "book_id": 2,
        "title": "1984",
        "author": "George Orwell",
        "genre": "Dystopian"
    },
    {
        "book_id": 3,
        "title": "The Hunger Games",
        "author": "Suzanne Collins",
        "genre": "Young Adult"
    }
]


## 2.1.3 要求模型检查是否满足条件

如果任务包含不一定能满足的假设（条件），我们可以告诉模型先检查这些假设，如果不满足，则会指出并停止执行后续的完整流程。您还可以考虑可能出现的边缘情况及模型的应对，以避免意外的结果或错误发生。

在如下示例中，我们将分别给模型两段文本，分别是制作茶的步骤以及一段没有明确步骤的文本。我们将要求模型判断其是否包含一系列指令，如果包含则按照给定格式重新编写指令，不包含则回答“未提供步骤”。

In [16]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let the tea steep for a few minutes.
Step 5 - Remove the tea bag.
Step 6 - Add sugar or milk to taste.
Step 7 - Enjoy your delicious cup of tea.


In [17]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:
Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


In [18]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。
第二步 - 拿一个杯子并把茶包放进去。
第三步 - 把烧开的水倒在茶包上。
第四步 - 等待几分钟，让茶叶浸泡。
第五步 - 取出茶包。
第六步 - 如果愿意，可以加糖或牛奶调味。
第七步 - 尽情享受一杯美味的茶。


In [19]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


## 2.1.4 提供少量示例（少样本提示词，Few-shot prompting）

即在要求模型执行实际任务之前，提供给它少量成功执行任务的示例。

例如，在以下的示例中，我们告诉模型其任务是以一致的风格回答问题，并先给它一个孩子和祖父之间的对话的例子。孩子说，“请教我何为耐心”，祖父用下述风格的隐喻来回答。由于我们已经告诉模型要以一致的语气回答，因此现在我们问“请教我何为韧性”，由于模型已经有了这个少样本示例( few-shot example )，它将以类似的语气回答下一个任务。

In [20]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The tallest trees withstand the strongest storms; the brightest stars shine through the darkest nights; the strongest hearts endure the toughest trials.


In [21]:
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 就像大树在风雨中屹立不倒，韧性是一种坚韧不拔的品质。要学会面对困难和挑战，坚持不懈地追求目标，即使遇到困境也要坚强地挺过去。只有经历风雨的洗礼，才能变得更加坚强和有韧性。所以，要学会坚持不懈，永不放弃，这样你就能拥有真正的韧性。


# 2.2 给模型时间去思考
如果您发现模型推理过程过于匆忙，导致得出了错误的结论，那么您应该尝试重新构思 Prompt ，要求模型在提供最终答案之前开展思维链，或进行一系列相关推理（a chain or series of relevant reasoning）。换句话说，如果您给模型一个在短时间内或用少量文字无法完成的复杂任务，它的输出结果就容易出错。这种情况对人来说也是类似：如果您要求某人完成复杂的数学问题，又不给足够时间计算出答案，他们也可能会犯错误。因此，在这些情况下，您应该指示模型花更多时间思考问题，让它在任务上花费更多计算资源。

## 2.2.1 指定完成任务所需的步骤

接下来我们将通过给定一个复杂任务，给出完成该任务的一系列步骤，来展示这一策略的效果。

首先我们描述了杰克和吉尔的故事，并给出提示词执行以下操作：首先，用一句话概括三个反引号限定的文本。第二，将摘要翻译成法语。第三，在法语摘要中列出每个名称。第四，输出包含以下键的 JSON 对象：法语摘要和人名个数。要求输出以换行符分

In [8]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet is indeed 450x + 100,000.


In [9]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)

学生的解决方案是正确的。总费用的计算是正确的，即总费用为450x+100,000美元。


In [10]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = $100x + $250x + $100,000 + $10x
Total cost = $360x + $100,000

The total cost for the first year of operations as a function of the number of square feet is $360x + $100,000.
```
Is the student's solution the same as actual solution just calculated:
```
No
```
Student grade:
```
incorrect


In [11]:
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：
首先，自己解决问题，解决问题时列数学表达式。
然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
**学生的计算结果：学生的计算结果文本
实际计算结果：实际计算结果文本
学生的计算结果和实际计算结果是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否**
学生的成绩：正确或不正确

问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

费用：
1. 土地费用：100美元/平方英尺
2. 太阳能电池板费用：250美元/平方英尺
3. 维护费用：固定10万美元+每平方英尺10美元

首年总费用：
总费用 = 土地费用 + 太阳能电池板费用 + 维护费用
总费用 = 100x + 250x + (100,000 + 10x)
总费用 = 350x + 100,000

学生的计算结果：450x + 100,000
实际计算结果：350x + 100,000
学生的计算结果和实际计算结果是否相同：否
学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确


In [12]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech toothbrush designed to provide a superior cleaning experience. It features a slim and sleek design that is easy to hold and maneuver, making it ideal for reaching all areas of the mouth.

The toothbrush is equipped with smart technology that tracks your brushing habits and provides real-time feedback to help you improve your oral hygiene routine. It also has a built-in timer and pressure sensor to ensure that you are brushing for the recommended two minutes and applying the right amount of pressure.

The bristles of the AeroGlide UltraSlim Smart Toothbrush are made from a durable and hygienic material called Boie's patented BPA-free thermoplastic elastomer. This material is gentle on the gums and teeth, yet effective at removing plaque and debris.

Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie is a cutting-edge dental tool that combines advanced technology with a sleek and user-friendly design to help you achie

In [13]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息
"""
response = get_completion(prompt)
print(response)

Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款智能牙刷，具有超薄设计。这款牙刷采用先进的技术，可以提供高效的清洁和护理，帮助用户保持口腔健康。AeroGlide UltraSlim Smart Toothbrush具有智能感应功能，可以根据用户的刷牙习惯和需求进行个性化的清洁模式设置。此外，这款牙刷还配备了长续航的电池，方便用户在旅行或出行时使用。总的来说，Boie公司的AeroGlide UltraSlim Smart Toothbrush是一款高品质、智能化的牙刷产品，适合追求高效清洁和便捷使用体验的消费者。


In [14]:
prompt = f"""
告诉我 Boie 公司生产的 AeroGlide UltraSlim Smart Toothbrush 的相关信息\
    并提供相关的引用信息
"""
response = get_completion(prompt)
print(response)

Boie公司生产的AeroGlide UltraSlim Smart Toothbrush是一款智能牙刷，具有超薄设计和高效的清洁功能。该牙刷采用先进的技术，能够提供深层清洁和舒适的刷牙体验。此外，AeroGlide UltraSlim Smart Toothbrush还配备了智能功能，可以帮助用户更好地监控和改善口腔健康。

引用信息：
Boie官方网站：https://boieusa.com/products/aeroglide-ultraslim-smart-toothbrush

通过访问Boie官方网站，您可以了解更多关于AeroGlide UltraSlim Smart Toothbrush的详细信息和购买方式。
